In [2]:
import pandas as pd

In [3]:
# Load dataset
df = pd.read_csv("email.csv")

In [6]:
# Check unique values in Category column
print("Unique values in Category column:", df["Category"].unique())

# Filter only valid labels (ham or spam)
df = df[df["Category"].isin(["ham", "spam"])]

# Convert labels (0 = ham, 1 = spam)
df["label"] = df["Category"].map({"ham": 0, "spam": 1})


Unique values in Category column: ['ham' 'spam']


In [7]:
# Ensure there are no NaN values after mapping
print("NaN values in label column:", df["label"].isnull().sum())

NaN values in label column: 0
